<a target="_blank" href="https://colab.research.google.com/github/tds2023-24/course/blob/main/notebooks/05_Deep_Learning_Tabular.ipynb
">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<div class='bar_title'></div>

*Practical Data Science*

# Deep Learning on Tabular Data

Gunther Gust<br>
Chair of Enterprise AI <br>
Data Driven Decisions Group &<br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="images/d3.png" style="width:20%; float:left;" />

<img src="images/CAIDASlogo.png" style="width:20%; float:left;" />

__Credits__

<img src="https://images-na.ssl-images-amazon.com/images/I/516YvsJCS9L._SX379_BO1,204,203,200_.jpg" width="200" align="right"/>

In the next lectures we will dive into Deep Learning using ressources from the book of 
**Jeremy Howard and Sylvian Gugger: "Deep Learning for Coders with Fastai and PyTorch: AI Applications without a PhD." (2020).**

It's freely available as interactive [Jupyter Notebook](https://github.com/fastai/fastbook) 

Materials also taken from:
- https://github.com/fastai/fastbook/blob/master/09_tabular.ipynb
- https://www.fast.ai/2018/04/29/categorical-embeddings/
- https://confusedcoders.com/data-science/deep-learning/how-to-apply-deep-learning-on-tabular-data-with-fastai 

## Motivation

At the end of 2015, the [Rossmann sales competition](https://www.kaggle.com/c/rossmann-store-sales) ran on Kaggle. Competitors were given a wide range of information about various stores in Germany, and were tasked with trying to predict sales on a number of days. The goal was to help the company to __manage stock__ properly and be able to __satisfy demand without holding unnecessary inventory.__ The official training set provided a lot of information about the stores. It was also permitted for competitors to use additional data, as long as that data was made public and available to all participants.

One of the gold medalists used deep learning, in one of the __earliest known examples__ of a __state-of-the-art deep learning tabular model.__ Their method involved far __less feature engineering__, based on domain knowledge, than those of the other gold medalists. The paper, ["Entity Embeddings of Categorical Variables"](https://arxiv.org/abs/1604.06737) describes their approach. The authors state:

> Entity embedding not only **reduces memory usage** and **speeds up neural networks compared with one-hot encoding**, but more importantly by **mapping similar values close to each other in the embedding space** it reveals the intrinsic properties of the categorical variables... 

> [It] is especially useful for datasets with lots of **high cardinality features**, where other methods tend to overfit... As entity embedding defines a distance measure for categorical variables it can be used for visualizing categorical data and for data clustering.

### Categorical Embedding Examples

We will have a look at the examples from the paper ["Entity Embeddings of Categorical Variables"](https://arxiv.org/abs/1604.06737) 

**State embeddings and map**

<img alt="State embeddings and map" width="50%" caption="State embeddings and map (courtesy of Cheng Guo and Felix Berkhahn)" id="state_emb" src="https://raw.githubusercontent.com/fastai/fastbook/master/images/att_00015.png">

On the left is a plot of the embedding matrix for the possible values of the `State` category. For a categorical variable we call the possible values of the variable its "levels" (or "categories" or "classes"), so here one level is "Berlin," another is "Hamburg," etc. On the right is a map of Germany. The actual physical locations of the German states were not part of the provided data, yet the model itself learned where they must be, based only on the behavior of store sales!

**Store Distances**

The distance between store embeddings against the actual geographic distance between the stores - they match very closely!

<img alt="Store distances" width="50%" caption="Store distances (courtesy of Cheng Guo and Felix Berkhahn)" id="store_emb" src="https://raw.githubusercontent.com/fastai/fastbook/master/images/att_00016.png">

**Date Embedding**

Days and months that are near each other on the calendar ended up close as embeddings too.

<img alt="Date embeddings" width="50%" caption="Date embeddings" id="date_emb" src="https://raw.githubusercontent.com/fastai/fastbook/master/images/att_00017.png">

How can we train such embeddings?

## Artificial Neural Networks

__What are neural networks?__

- Biological neural networks have interconnected neurons with dendrites that receive inputs, then based on these inputs they produce an output signal through an axon to another neuron
- Artificial Neural Networks (ANN) are a machine learning framework that attempts to mimic the learning pattern of natural biological neural networks
- The creation of ANN begins with the most basic form, a single perceptron

<img src="./images/05/DALL·E 2023-10-07 15.23.07.png" width="30%"/>


### The Perceptron

Developed by Frank Rosenblatt in 1957
- Perceptrons have one or more weighted inputs, a bias, an activation function, and a single output
- A perceptron receives inputs, multiplies them by some weight, and then passes them into an activation function to produce an output
- The key idea is to “fire” / activate the neuron only if a sufficiently strong input signal is detected

<img src="https://miro.medium.com/max/2870/1*n6sJ4yZQzwKL9wnF5wnVNg.png" width="50%"/>


__Different Activation Functions and their Graphs__

<img src="https://miro.medium.com/max/1200/1*ZafDv3VUm60Eh10OeJu1vw.png" width="50%"/>

[Image Source](https://medium.com/@shrutijadon10104776/survey-on-activation-functions-for-deep-learning-9689331ba092)

ReLU is widely used as an activation function because its derivative is easy to compute during backpropagation, making it both simple and computationally efficient.

## Multi-layer Perceptron aka. Neural Network

A MLP is composed of multiple layers of perceptrons 

<img src="https://camo.githubusercontent.com/8a0c06dc8a564e1e7732169d97e2685eafc2b98b5f4915f4c0cb6bf2c1649a37/68747470733a2f2f7777772e64726f70626f782e636f6d2f732f717334746f6a763575356834386c662f6d756c74696c617965725f70657263657074726f6e2e706e673f7261773d31" style="width:80%" />

[Image Source](https://github.com/PetarV-/TikZ/tree/master/Multilayer%20perceptron)

__Layers of a MLP__

- Initial layer = input layer which is fed by the feature inputs
- Last layer = output layer which creates the resulting outputs
- Any layers in between are known as hidden layers because they do not directly “observe” the feature inputs or outputs

__Universal approximation theorem__

From Wikipedia:

_"In the mathematical theory of artificial neural networks, the universal approximation theorem states that a feed-forward network with __a single hidden layer__ containing a finite number of neurons can approximate any continuous function [...] when given appropriate parameters; however, it does not touch upon __the algorithmic learnability of those parameters__."_

__Loss function__

The loss function $L$ is a way for a model to measure how well it’s doing. Imagine you have a goal, like correctly classifying pictures of cats and dogs. Each time the model makes a guess, the loss function calculates how far off that guess is from the true answer. It gives a number (called "loss" or "error") that __tells you how bad the guess was.__

The goal of __training__ the model is to make the loss as small as possible by __minimizing the loss function.__

For more explanation see e.g. [this video](https://youtu.be/QBbC3Cjsnjg). Common choices are __MeanSquaredError (MSE)__ loss for regression tasks and __CrossEntropy loss__ for classification tasks, see [here](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html) for the math. 


### Training Neural Networks

Learning is adjustment of the weights of the connections between perceptrons according to some modification rule. 

- The Backpropagation algorithm searches for weight values that minimize the total error of the network over the set of training examples

It consists of the repeated application of the following two passes.

- __Forward pass__: in this step the network is activated on one example and the error of (each neuron of) the output layer is computed
- __Backward pass__: in this step the network error is used for updating the weights

__Forward and Backward passes__

<img src="https://miro.medium.com/max/3108/1*6q2Rgd8W9DoCN9Wfwc_9gw.png" style="width:60%" />

[Image Source](https://medium.com/datathings/neural-networks-and-backpropagation-explained-in-a-simple-way-f540a3611f5e)

## Core Ideas of Backpropagation

### 1. Compute the gradient for each parameter

Backpropagation is built on the idea that we can compute __how the loss $L$ changes__ when we change a single __weight__ $w$. This is captured by the partial derivative

$$
\frac{\partial L}{\partial w}
$$

which tells us whether increasing $w$ makes the loss larger or smaller, and by how much. Backpropagation efficiently computes these gradients for __*all* parameters__ by applying the __chain rule__ through the network.



### 2. Update each parameter using the gradient

Once the gradient is known, each weight is updated using gradient descent:

$$
w \leftarrow w - \eta \frac{\partial L}{\partial w}
$$

This moves the parameter in the direction that reduces the loss. Small steps are taken repeatedly until the network learns to produce better predictions.


For an illustrative explanation of how to train a neural network using backpropagation see [this](https://www.youtube.com/watch?v=iyn2zdALii8&list=PLblh5JKOoLUIxGDQs4LFFD--41Vzf-ME1&index=6&ab_channel=StatQuestwithJoshStarmer) and related videos. 

## Implementing a Multi Layer Perceptron

We will work with the same dataset as in the last lecture, a sample of the adult dataset which has some census information on individuals. Again, we'll use it to train a model to predict whether salary is greater than $50k or not.

### Load packages

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

### Load data set

In [ ]:
file_path = 'https://raw.githubusercontent.com/GuntherGust/tds2_data/main/data/adult.csv'
adult_data = pd.read_csv(file_path)
adult_data = adult_data.assign(salary=(adult_data['salary']=='>=50k').astype(int))
y = adult_data['salary']
X = adult_data.drop(columns=['salary'])

### Split data set

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25, random_state = 0)

### Impute missing values (we will omit the categorical features here)

In [ ]:
simple_imputer = SimpleImputer()
numCols = X.select_dtypes(['int', 'float']).columns.to_list()
train_X_num = pd.DataFrame(simple_imputer.fit_transform(train_X[numCols]), columns=numCols, index=train_X.index)
val_X_num = pd.DataFrame(simple_imputer.transform(val_X[numCols]), columns=numCols, index=val_X.index)

### Standardize numeric features

In [ ]:
scaler = StandardScaler()
train_X_num_standardized = pd.DataFrame(scaler.fit_transform(train_X_num), columns=numCols, index=train_X.index)
val_X_num_standardized = pd.DataFrame(scaler.transform(val_X_num), columns=numCols, index=val_X.index)

### Train model

In [ ]:
model = MLPClassifier()
model.fit(train_X_num_standardized, train_y)

```MLPClassifier()``` allows to create arbitrarily-sized neural networks. For example:

- ```MLPClassifier(hidden_layer_sizes=())``` A simple perceptron (no hidden layer)
- ```MLPClassifier(hidden_layer_sizes=(10,))``` Creates a NN with one hidden layer with 10 neurons
- ```MLPClassifier()``` Creates a NN with one hidden layer with 100 neurons (default setting)
- ```MLPClassifier(hidden_layer_sizes=(128, 64, 32, 16))``` A deeper 4-layer network: Input → 128 → 64 → 32 → 16 → Output

Many __more important parameters,__ such as _activation functions_, _optimizer_, etc., can be configured. Check out the [official documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html).

### Evaluate predictions

In [ ]:
preds = model.predict(val_X_num_standardized)
accuracy_score(val_y, preds)

__Advantages of Multi-layer Perceptrons__

- Capability to learn non-linear models.
- Capability to learn models in real-time (on-line learning) using `partial_fit`

__The disadvantages of Multi-layer Perceptrons__
- MLP with hidden layers have a non-convex loss function where there exists more than one __local minimum.__ Therefore different random weight initializations can lead to different validation accuracy.
- MLP requires tuning a number of __hyperparameters__ such as the number of hidden neurons, layers, and iterations.
- MLP is sensitive to __feature scaling.__

[from scikit-learn](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)

__Is this already deep learning?__

From Wikipedia: 

_"Deep learning [...] uses multiple layers to progressively extract higher level features from the raw input. For example, in image processing, lower layers may identify edges, while higher layers may identify the concepts relevant to a human such as digits or letters or faces."_ 


## Exercises - XOR Problem

The XOR (exclusive OR) is a logical operation that takes two binary outputs and returns TRUE if exactly one of them is TRUE. For example:
- f(1, 0) = 1
- f(1, 1) = 0
- f(0, 1) = 1
- f(0, 0) = 0

___Background:___ The XOR problem has a famous history with regard to neural networks:

- XOR is not linearly separable, so a single-layer perceptron cannot learn it.
- This limitation, highlighted by Minsky & Papert (1969), led to broad skepticism about neural networks.
- Adding a hidden layer allows a neural network to solve XOR, showing the power of multilayer models.
- The success of backpropagation in training such multilayer networks helped spark the modern deep learning era.

We provide you a XOR-like dataset with 80 samples and 2 labels as below:

In [ ]:
# ONLY RUN & DO NOT MODIFY
import numpy as np
import matplotlib.pyplot as plt

# Create XOR dataset
np.random.seed(0)

# Create points around the corners of the XOR problem
X_corners = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_corners = np.array([0, 1, 1, 0])

# Generate additional random samples in each quadrant
num_samples_per_corner = 20
X_samples = []
y_samples = []

for x, y in X_corners:
    # Generate samples around each corner
    X_samples.append(np.random.rand(num_samples_per_corner, 2) * 0.2 + np.array([x - 0.1, y - 0.1]))
    y_samples.append(np.full((num_samples_per_corner,), y_corners[np.where((X_corners == [x, y]).all(axis=1))[0][0]]))

X_samples = np.vstack(X_samples)
y_samples = np.concatenate(y_samples)

# Step 2: Split Data into Training and Validation Sets
# Ensure that each class is represented in both sets
indices = np.arange(len(X_samples))
np.random.shuffle(indices)

# Manually split to maintain distribution
train_indices = []
val_indices = []

for i in range(4):  # For each corner
    corner_indices = indices[i*num_samples_per_corner:(i+1)*num_samples_per_corner]
    train_indices.extend(corner_indices[:int(num_samples_per_corner * 0.7)])  # 70% for training
    val_indices.extend(corner_indices[int(num_samples_per_corner * 0.7):])    # 30% for validation

train_X_xor = X_samples[train_indices]
train_y_xor = y_samples[train_indices]
val_X_xor = X_samples[val_indices]
val_y_xor = y_samples[val_indices]

plt.scatter(X_samples[y_samples == 0][:, 0], X_samples[y_samples == 0][:, 1], color='red', label='Class 0')
plt.scatter(X_samples[y_samples == 1][:, 0], X_samples[y_samples == 1][:, 1], color='blue', label='Class 1')
plt.title('XOR Dataset')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

Your tasks are:

1. Define an MLP with one hidden layer with one neuron and fit this dataset. Run 1000 iteration.
2. Define an MLP with one hidden layers with eight neurons and fit this dataset. Run 1000 iteration.
3. Look at the decision boundary, and try to explain the results. Hint: check which model provides better decision boundary, and why it is the case.

Hint: use `train_X_xor, train_y_xor` for training, and `val_X_xor`, `val_y_xor` for testing.

In [ ]:
# YOUR CODE HERE
# define, fit, and predict the first model with one hidden layer. 
# Hint 1: MLPClassifier(hidden_layer_sizes=(number_layer, ), max_iter=num_iterations)
# Hint 2: the accuracy should never reach 80% 
mlp_hidden_1 = ... # define the model
mlp_hidden_1.fit(...) # fit the model
preds_hidden_1 = ... # predict the model
accuracy_score(...) # compute the accuracy

In [ ]:
# YOUR CODE HERE
# define, fit, and predict the first model with 8 hidden layers. 
# Hint 1: MLPClassifier(hidden_layer_sizes=(number_layer, ), max_iter=num_iterations)
# Hint 2: the accuracy can reach 100%. Try to re-run until you get accuracy=1.0
mlp_hidden_8 = ... # define the model
mlp_hidden_8.fit(...) # fit the model
preds_hidden_8 = ... # predict the model
accuracy_score(...) # compute the accuracy

Now, we plot the decision boundary of 2 cases:

In [ ]:
# RUN & DO NOT MODIFY
def plot_decision_boundary(model, val_X, val_y, num_layer):
    xx, yy = np.meshgrid(np.linspace(-0.5, 1.5, 100), np.linspace(-0.5, 1.5, 100))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])  # Predicting over the grid
    Z = Z.reshape(xx.shape)

    # Plotting decision boundary and validation samples
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')

    # Plot validation points
    plt.scatter(val_X[val_y == 0][:, 0], val_X[val_y == 0][:, 1], color='darkred', label='Val Class 0', marker='x')
    plt.scatter(val_X[val_y == 1][:, 0], val_X[val_y == 1][:, 1], color='darkblue', label='Val Class 1', marker='x')

    plt.title(f'Decision Boundary of MLP with {num_layer} Hidden Layer')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.grid()
    plt.show()

### Decision Boundary of MLP with 1 hidden layer

In [ ]:
# Run this after finishing the above tasks
plot_decision_boundary(mlp_hidden_1, val_X_xor, val_y_xor, 1)

### Decision Boundary of MLP with 8 hidden layers

In [ ]:
# Run this after finishing the above tasks
plot_decision_boundary(mlp_hidden_8, val_X_xor, val_y_xor, 8)

## Deep Learning on Tabular Data with *fast.ai*

**The Mission of [fast.ai](https://www.fast.ai/about.html): Making neural nets uncool again**

Deep learning is transforming the world. We are making deep learning easier to use and getting more people from all backgrounds involved through our:

- [free courses for coders](http://course.fast.ai/)
- software library: [fastai for PyTorch](http://docs.fast.ai/)
- cutting-edge research
- community

The world needs everyone involved with AI, no matter how unlikely your background.

First, let's import everything we need for the tabular application.

In [ ]:
#!pip install -Uqq fastai  # upgrade fastai on colab
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split

`from <module> import *` means “I want access to all the names in <module> that I’m meant to have access to”

### *fast.ai* Datasets

Tabular data usually comes in the form of a delimited file (such as .csv) containing variables of different kinds: text/category, numbers, and perhaps some missing values. 

*Fast.ai's* [external data functions](https://docs.fast.ai/data.external.html) provides several useful datasets that we might be interested in using in our models.

We will work with the same dataset as in the last lecture, a sample of the __adult dataset__ which has some census information on individuals. Again, we'll use it to train a model to predict whether __salary is greater than \$50k__ or not.

In [ ]:
path = untar_data(url=URLs.ADULT_SAMPLE)
path.ls()

`untar_data()`downloads a dataset from `url` and unpacks it to `path`.

In [ ]:
df = pd.read_csv(path/'adult.csv')
df.head()

Here all the information that will form our input is in the 14 first columns, and the dependent variable is the last column. We will split our input between two types of variables: categorical and continuous.

### From data to dataloaders

fastai uses [data loaders](https://docs.fast.ai/data.load.html) to get the data ready for training.

A data loader usually combines a __dataset and a sampler,__ and provides an __iterable__ over the given dataset. [fastai](https://docs.fast.ai/data.load.html) includes a replacement for [Pytorch's DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) which is largely API-compatible, and adds a lot of useful functionality and flexibility.

How do we create a data loader?

__Categorical and continuous variables__

- **Categorical variables** (like workclass or education) will be replaced by a category - a unique id that identifies them - before they are passed through an embedding layer.
- **Continuous variables** (like age) will be normalized and then directly fed to the model.

We can specify our categorical and continuous column names, as well as the name of the dependent variable:

In [ ]:
y_names = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']

__Tabular data preprocessing__

fast.ai contains classes that define [transformations](https://docs.fast.ai/tabular.core.html#TabularProc) for preprocessing dataframes of tabular data. Preprocessing includes things like
- `FillMissing`: filling missing values for continuous variables (default fill strategy: median). Missing values in categorical columns are handled automatically by `Categorify`, which treats ```NaN, None, ''``` as a separate category.
- `Categorify`: replacing non-numerical variables by categories (each value obtains a category id, as in label encoding). 
- `Normalize:` normalizing continuous variables. (Categorical columns are __not__ normalized. Category indices are passed directly into an embedding layer)

We can define a list of Transforms that will be applied to our variables. Here we transform all categorical variables into categories. We also replace missing values for continuous variables by the median column value and normalize those.

In [ ]:
procs = [FillMissing, Categorify, Normalize]

Note that the __ordering__ of the tranformations matters. The typical sequence is to first handle missing values, then categorize categorical variables (including newly created categorial variables that indicate missingness), and finally normalize continuous variables. 

__Training and validation sets__

To split our data into training and validation sets, we use valid indexes

In [ ]:
train_idx, valid_idx = train_test_split(df.index, test_size=0.25, random_state = 0)
train_idx[:5] 

__Creating the DataLoader__

Now we're ready to pass this information to a [TabularDataLoader](https://docs.fast.ai/tabular.data.html#TabularDataLoaders) to create the DataLoaders that we'll use for training. We will learn the details of `DataLoaders` class in the next lecture.

In [ ]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, 
                                  y_names=y_names,
                                  cat_names = cat_names,
                                  cont_names = cont_names,
                                  valid_idx=valid_idx,
                                  procs = procs,
                                 bs=64)

In [ ]:
dls.cat_names

We can grab a mini-batch of data and take a look. `show_batch` shows a batch of data in a convenient way:

__Note__: As we pick out batches randomly, the output of `show_batch` may not correspond to the output below. 

In [ ]:
dls.show_batch()

After being processed, the categorical variables are replaced by ids and the continuous variables are normalized. The codes corresponding to categorical variables are all put together, as are all the continuous variables.

But how does the data exactly look like for our model? Let's have a look:


In [ ]:
batch = dls.one_batch()
batch

__Question__: Can you explain the individual components of the previous outputs?
- I.e. what is contained in the individual tensors? (Hint: look at the dimensions of the tensors)

## Defining a Model

Once we have our data ready in `DataLoaders`, we just need to create a model to then define a Learner and start training. 

This is typically composed of following steps :

1. __Create Learner__: Create an appropriate learner for data. A learner creates a neural network for us.
2. __Find the learning rate__: We need to find a suitable learning rate for our training
3. __Fit the model__

__Create Learner__

The fastai library has a flexible and powerful `TabularModel`. The `tabular_learner` will automatically create a `TabularModel` suitable for your data and infer the right loss function. 

In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

Let's print a summary of the model.

In [ ]:
learn.summary()

### Visualizing the model graph with TensorBoard

TensorBoard is a tool for providing the measurements and visualizations needed during the machine learning workflow. It enables 

- tracking experiment metrics like loss and accuracy, 
- visualizing the model graph, 
- projecting embeddings to a lower dimensional space, 
- and much more.

Let's load the TensorBoard notebook extension

In [ ]:
%load_ext tensorboard

The SummaryWriter class is your main entry to log data

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Create a writer (for later logging the model and visualizing it in tensorboard) in the directory "tb-tabular"
writer = SummaryWriter('tb-tabular')

Write model architecture:

In [ ]:
# fetch one batch of data
batch = dls.one_batch() #alternatively: next(iter(dls.train))
# load the model to the cpu and pass some instances of the batch through it (this is necessary for the writer to log the computational graph)
writer.add_graph(learn.model.cpu(), batch[:2])

Start TensorBoard

In [ ]:
# Start tensorboard providing the previously specified directory
%tensorboard --logdir tb-tabular

Instead of displaying tensorboard in the jupyter notebook cell, it may be better to open it in a new browser tab using "localhost:<port>", e.g "localhost:6006" in the address bar.


For an additional example on how to analyze model architectures with tensorboard, see e.g. [this video](https://youtu.be/9SdLOcGnebU?t=567). 

### Train the model

__Find the learning rate__

In [ ]:
learn.lr_find()

The learning rate finder will do a __mock training__ by going over a large range of learning rates, then __plot them against the losses.__ We typically find the point where the slope is steepest. 

We will learn more about the Learning Rate finder and 1cycle policy, but here are some excellent resources that offer explanation: 
* https://fastai1.fast.ai/callbacks.one_cycle.html
* https://sgugger.github.io/the-1cycle-policy.html



__Fit the model__ based on selected learning rate

In [ ]:
learn.fit_one_cycle(5, lr_max=0.001)

## Model Evaluation

We can then have a look at some predictions:

In [ ]:
learn.show_results()

__Get predictions__

We can use the `Learner.predict` method to get predictions. In this case, we need to pass the row of a dataframe that has the same names of categorical and continuous variables as our training or validation dataframe.

In [ ]:
row, clss, probs = learn.predict(df.iloc[0])

In [ ]:
df.iloc[0]

In [ ]:
clss, probs

**Calculate performance metrics**

In [ ]:
learn.validate()

... shows the validation loss and the validation metric (accuracy). 
We can manually compute this as:

In [ ]:
probs, val_y = learn.get_preds(ds_idx=1)
preds = np.argmax(probs, axis=1)
accuracy_score(val_y, preds)

To get prediction on a new dataframe, you can use the `test_dl` method of the DataLoaders. That dataframe does not need to have the dependent variable in its column.

In [ ]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

__Show rows result of predictions on the dataset__

In [ ]:
learn.get_preds(dl=dl)

So there is a scope of improving the deep learning model here. However this is not bad at all, without any feature engineering and network tuning.

## Embeddings for Categorical Variables

A key technique to making the most of deep learning for tabular data is to use embeddings for your categorical variables. This approach allows for __relationships between categories__ to be captured.

Examples:
- Saturday and Sunday may have similar behavior, and maybe Friday behaves like an average of a weekend and a weekday. 
-  Similarly, for zip codes, there may be patterns for zip codes that are geographically near each other, and for zip codes that are of similar socio-economic status.

### Applying Embeddings for Categorical Variables

When working with categorical variables, we will represent each category by a vector of floating point numbers (the values of this representation are learned as the network is trained).

For instance, a 4-dimensional version of an embedding for day of week could look like:

__Sunday	 [.8, .2, .1, .1]__<br>
__Monday	[.1, .2, .9, .9]__<br>
__Tuesday	[.2, .1, .9, .8]__

Here, Monday and Tuesday are fairly similar, yet they are both quite different from Sunday. 

Again, this is a toy example. In practice, our neural network would learn the best representations for each category while it is training, and each dimension (or direction, which doesn’t necessarily line up with ordinal dimensions) could have multiple meanings. Rich relationships can be captured in these distributed representations. 



The choice of the dimensions of the embeddings depends on many factors, such as training set size, complexity of the learning task and computational resources available.

Fast.ai (see [emb_sz_rule](https://docs.fast.ai/tabular.model.html#emb_sz_rule)) applies the following rule of thumb: 
- __Embedding size = min(50, round(1.6 * n_categories^0.56))__


### Visualizing Embeddings with Tensorboard

Export embeddings:

In [ ]:
for i, emb in enumerate(learn.model.embeds):
    emb_name = learn.dls.cat_names[i]
    writer.add_embedding(emb.weight.data, metadata=learn.dls.classes[emb_name],
                         global_step=i, tag=emb_name)

Finally, start tensorboard

In [ ]:
%tensorboard --logdir tb-tabular

___Colab Workaround___

In Colab the dynamic tensorboard plugin isn’t supported yet, but you can still access the data and visualize the embeddings somewhere else: 

1. Download the desired embedding file (*tensors.tsv*) and metadata 
2. Upload the files on the official Tensorflow [Embedding Projector](https://projector.tensorflow.org/)

On the [webpage](https://projector.tensorflow.org/) of the Embedding Projector, you can also explore example word and image embeddings. Feel free to try!

## Mentimeter

### Conclusion

For analyzing time series and tabular data, deep learning has recently been making great strides. However, deep learning is generally used as part of an **ensemble of multiple types** of model. 

- If you already have a system that is using random forests or gradient boosting machines, then switching to or adding deep learning may not result in any dramatic improvement. 
- Deep learning does greatly increase the variety of columns that you can include
    - columns containing natural language (book titles, reviews, etc.), 
    - high-cardinality categorical columns (i.e., something that contains a large number of discrete choices, such as zip code or product ID). 
- Deep learning models generally take longer to train than random forests or gradient boosting machines, although this is changing thanks to libraries such as [RAPIDS](https://rapids.ai/), which provides GPU acceleration for the whole modeling pipeline.

## Final Notes

* The material of this lecture is very advanced if you are not familiar to neural networks
* You may be able to complete the course without diving deeper into them
* However, to get most out of this class (and prepare yourself to become a data scientists), we highly recommend to check out the additional material provided. In particular:
    * the materials and courses offeerd by [fastai](https://www.fast.ai/about.html)
    * The book: Jeremy Howard and Sylvian Gugger (2020), "Deep Learning for Coders with Fastai and PyTorch: AI Applications without a PhD." (2020). It's freely available as interactive [Jupyter Notebook](https://github.com/fastai/fastbook) 
    * Also Andrew NGs [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning?utm_medium=sem&utm_source=gg&utm_campaign=B2C_EMEA_deep-learning_deeplearning-ai_FTCOF_specializations_country-DE&campaignid=20416373453&adgroupid=155810822830&device=c&keyword=deep%20learning%20specialization&matchtype=b&network=g&devicemodel=&adposition=&creativeid=667829385242&hide_mobile_promo&gclid=Cj0KCQiAr8eqBhD3ARIsAIe-buNiwv5lVHgaI7bjlWFq52LxRgQNdlvNCevnV_33f_ZX8Dc4wKwLIUEaAiTCEALw_wcB) is a great course for learning more about the theory behind neural networks


<img src="images/d3.png" style="width:50%; float:center;" />